In [ ]:
#Attaching drive
from google.colab import drive
drive.mount('/content/drive')

# **People Detection Function**

In [ ]:
import numpy as np
import cv2

def detect_people(frame, net, ln, personIdx=0):

	MIN_CONF = 0.3  # confidence percentage
	NMS_THRESH = 0.3 #confidence/non maxima threshold


	(H, W) = frame.shape[:2] #grabbing frames
	results = []

	blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),swapRB=True, crop=False) #create blob from every frame
	net.setInput(blob) # passing it to yolo net
	layerOutputs = net.forward(ln)

	# initialize our lists of detected bounding boxes, centroids, and confidences, respectively
	boxes = []
	centroids = []
	confidences = []

	# loop over each of the layer outputs
	for output in layerOutputs:
		# loop over each of the detections
		for detection in output:
			# extract the class ID and confidence
			scores = detection[5:]
			classID = np.argmax(scores)
			confidence = scores[classID]

			# Checking detetction for person
			if classID == personIdx and confidence > MIN_CONF:
				# scale the bounding box coordinates back relative to
				# the size of the image, keeping in mind that YOLO
				# actually returns the center (x, y)-coordinates of
				# the bounding box followed by the boxes' width and
				# height
				box = detection[0:4] * np.array([W, H, W, H])
				(centerX, centerY, width, height) = box.astype("int")

				# calculating centroid
				x = int(centerX - (width / 2))
				y = int(centerY - (height / 2))

				# update our list of bounding box coordinates,centroids, and confidences
				boxes.append([x, y, int(width), int(height)])
				centroids.append((centerX, centerY))
				confidences.append(float(confidence))

	# apply non-maxima suppression to suppress weak, overlapping bounding boxes
	idxs = cv2.dnn.NMSBoxes(boxes, confidences, MIN_CONF, NMS_THRESH)


	if len(idxs) > 0: #at least one detection exists
		# loop over the indexes we are keeping
		for i in idxs.flatten():
			# extract the bounding box coordinates
			(x, y) = (boxes[i][0], boxes[i][1])
			(w, h) = (boxes[i][2], boxes[i][3])


			# update our results list to consist of the person
			# prediction probability, bounding box coordinates,
			# and the centroid
			r = (confidences[i], (x, y, x + w, y + h), centroids[i])
			results.append(r)

	return results

# **measuring distances between detected people**

In [ ]:
# import the necessary packages

from google.colab.patches import cv2_imshow
from scipy.spatial import distance as dist
import numpy as np
import argparse
import imutils
import cv2
import os

MIN_DISTANCE = 75 #distance in pixels
# construct the argument parse and parse the arguments
# these are command line argument that can be skipped or used while using the code from terminal
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--input", type=str, default="",
	help="path to (optional) input video file") #this is required to take input
ap.add_argument("-o", "--output", type=str, default="",
	help="path to (optional) output video file") #this is required to give output
ap.add_argument("-d", "--display", type=int, default=1,
	help="whether or not output frame should be displayed")  # making default value can skip frame to frame execution on fore hand and directly gives result
args = vars(ap.parse_args(["--input","/content/drive/MyDrive/major/project/test videos/6.avi","--output","result.avi","--display","1"]))

# inputing object category file
labelsPath = os.path.sep.join(["/content/drive/My Drive/major/project/yolo-coco/coco.names"])
LABELS = open(labelsPath).read().strip().split("\n")

# inputing the weights and cfg files for yolo
weightsPath = os.path.sep.join(["/content/drive/My Drive/major/yolov3.weights"])
configPath = os.path.sep.join(["/content/drive/My Drive/major/project/yolo-coco/yolov3.cfg"])

# load our YOLO object detector trained on COCO dataset
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

# determine only the *output* layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# initialize the video stream and pointer to output video file
vs = cv2.VideoCapture(args["input"] if args["input"] else 0) # here we can take input from camera directly as well and here 0 defines that input from camera
writer = None

# loop over the frames from the video stream
while True:
	# read the next frame from the file
	(grabbed, frame) = vs.read()

	# if the frame was not grabbed, then we have reached the end
	if not grabbed:
		break

	# resize the frame and then detect people
	frame = imutils.resize(frame, width=700)
	results = detect_people(frame, net, ln,personIdx=LABELS.index("person"))  # calling the function


	# initialize the set of indexes that violate the minimum social distance
	violate = set()

	# ensure there are *at least* two people detections (required in
	# order to compute our pairwise distance maps)
	if len(results) >= 2:
		# extract all centroids from the results and compute the
		# Euclidean distances between all pairs of the centroids
		centroids = np.array([r[2] for r in results])
		D = dist.cdist(centroids, centroids, metric="euclidean")

		# loop over the upper triangular of the distance matrix
		for i in range(0, D.shape[0]):
			for j in range(i + 1, D.shape[1]):
				if D[i, j] < MIN_DISTANCE: # evaluating and checing distance between two centroids
					#updating violating index
					violate.add(i)
					violate.add(j)

	# loop over the results
	for (i, (prob, bbox, centroid)) in enumerate(results):
		# extract the bounding box and centroid coordinates, then
		# initialize the color of the annotation
		(startX, startY, endX, endY) = bbox
		(cX, cY) = centroid
		color = (0, 255, 0)

		# if the index pair exists within the violation set, then
		# update the color
		if i in violate:
			color = (0, 0, 255)



		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2) #displaying box
		cv2.circle(frame, (cX, cY), 5, color, 1) 	# centroid display



  # draw the total number of people on the
	# output frame
	text = "Total Number of People Detected: {}".format(len(results))
	cv2.putText(frame, text, (5, frame.shape[0] - 60),
		cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 255, 255), 3)

	# draw the total number of social distancing violations on the
	# output frame
	text = "Social Distancing Violations: {}".format(len(violate))
	cv2.putText(frame, text, (10, frame.shape[0] - 25),
		cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 3)

	# check to see if the output frame should be displayed to our
	# screen
	if args["display"] > 0:
		# show the output frame
		cv2_imshow(frame)
		key = cv2.waitKey(1) & 0xFF

		# if the `q` key was pressed, break from the loop
		if key == ord("q"):
			break

	# if an output video file path has been supplied and the video
	# writer has not been initialized, do so now
	if args["output"] != "" and writer is None:
		# initialize our video writer
		fourcc = cv2.VideoWriter_fourcc(*"MJPG")
		writer = cv2.VideoWriter(args["output"], fourcc, 25,
			(frame.shape[1], frame.shape[0]), True)

	# if the video writer is not None, write the frame to the output
	# video file
	if writer is not None:
		writer.write(frame)

